Beat-n game with 10 pt limit

## Spud code

In [1]:
using Random
using DataFrames
using CSV
using DelimitedFiles
using Statistics
using Printf
using LinearAlgebra
using Dates

In [2]:
struct Spud
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
    a1::Int64
end

In [3]:
const ability_none = 999
const ability_imitative = 10
const ability_reciprocating = 20
const ability_romantic = 30
const ability_bibliophile = 40
const ability_melodious = 50
const ability_iconoclast = 90
const ability_intoxicating = 100
const ability_otaku = 110
const ability_sincere = 120
const ability_cowardly = 130


ability_name = Dict(
    ability_imitative => "Imitative*",
    ability_reciprocating => "Reciprocating*",
    ability_romantic => "Romantic*",
    ability_bibliophile => "Bibliophile*",
    ability_melodious => "Melodious*",
    ability_iconoclast => "Iconoclast*",
    ability_intoxicating => "Intoxicating*",
    ability_otaku => "Otaku*",
    ability_sincere => "Sincere*",
    ability_cowardly => "Cowardly*",
)

function fx_imitative(a::Spud, b::Spud)
    if b.h > a.h
        a = Spud(a.name, b.h, a.f, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_reciprocating(a::Spud, b::Spud)
    if b.f > a.f
        a = Spud(a.name, a.h, b.f, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_romantic(a::Spud, b::Spud)
    if b.p > a.l
        a = Spud(a.name, a.h, a.f, b.p, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_bibliophile(a::Spud, b::Spud)
    if b.r > a.l
        a = Spud(a.name, a.h, a.f, b.r, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_melodious(a::Spud, b::Spud)
    if b.p > b.r && b.p > b.s
        a = Spud(a.name, a.h, a.f, a.r, a.p + 3, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_iconoclast(a::Spud, b::Spud)
    v = sum([b.h,b.f,b.l,b.p,b.r,b.s] .== MXV)
    if v >= 3
        aa = [a.h, a.f, a.l, a.p, a.r, a.s]
        aa2 = map(x -> div(x, 4), aa)
        r = aa .+ aa2
        a = Spud(a.name, r[1], r[2], r[3], r[4], r[5], r[6], a.a1)
    end
    return [a, b]
end

function fx_intoxicating(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f, b.l, b.p, b.r, b.f, b.a1)
    return [a, b]
end

function fx_otaku(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f, b.l, b.p, b.h, b.s, b.a1)
    return [a, b]
end


function fx_sincere(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f + 2, b.f + 2, b.p, b.r, b.s, b.a1)
    return [a, b]
end

function fx_cowardly(a::Spud, b::Spud)
    if (a.p < b.p) || (a.r < b.r) || (a.s < b.s)
        a = Spud(a.name, a.h + 2, a.f - 2, a.l, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

ability_fx = Dict(
    ability_imitative => fx_imitative,
    ability_reciprocating => fx_reciprocating,
    ability_romantic => fx_romantic,
    ability_bibliophile => fx_bibliophile,
    ability_melodious => fx_melodious,
    ability_iconoclast => fx_iconoclast,
    ability_intoxicating => fx_intoxicating,
    ability_otaku => fx_otaku,
    ability_sincere => fx_sincere,
    ability_cowardly => fx_cowardly,
)

Dict{Int64, Function} with 10 entries:
  50  => fx_melodious
  20  => fx_reciprocating
  110 => fx_otaku
  10  => fx_imitative
  90  => fx_iconoclast
  30  => fx_romantic
  120 => fx_sincere
  130 => fx_cowardly
  40  => fx_bibliophile
  100 => fx_intoxicating

In [4]:
#const MXS = 100
const MXV = 9
const MNV = 1

# function cost(h::Int64, f::Int64, l::Int64, p::Int64, r::Int64, s::Int64)::Int64
#     #return h*(f+l-2*MNV) + f*(p+r+s-3*MNV) + h+f+l+p+r+s-6*MNV
#     return h*(f+l) + f*(p+r+s)
# end

# function cost(a::Spud)::Int64
#     if a.a1 == ability_none
#         return cost(a.h, a.f, a.l, a.p, a.r, a.s)
#     else
#         return cost(a.h, a.f, a.l, a.p, a.r, a.s) + ability_cost[a.a1](a)
#     end
# end

1

In [5]:
function compare_int_list(as::Vector{Int64}, bs::Vector{Int64}, tiebreaker::Int64 = 0)::Int64
    n = min(length(as), length(bs))
    for i in 1:n
        if as[i] != bs[i]
            return sign(as[i] - bs[i])
        end
    end
    return tiebreaker
end

function spud_h_seq(a::Spud)::Vector{Int64}
    return [a.h, a.s, a.r, a.p, a.l, a.f]
end

function spud_f_seq(a::Spud)::Vector{Int64}
    return [a.f, a.s, a.r, a.p, a.l, a.h]
end

function spud_l_seq(a::Spud)::Vector{Int64}
    return [a.l]
end

function spud_p_seq(a::Spud)::Vector{Int64}
    return [a.p, a.l]
end

function spud_r_seq(a::Spud)::Vector{Int64}
    return [a.r, a.f]
end

function spud_s_seq(a::Spud)::Vector{Int64}
    return [a.s, a.h]
end

function spud_utb_seq(a::Spud)::Vector{Int64}
    return [a.a1, a.h, a.f, a.l, a.p, a.r, a.s]
end

function eval_finds(a::Spud, b::Spud, tiebreaker::Int64 = 0)::Int64
    ev = compare_int_list(spud_f_seq(a), spud_h_seq(b), tiebreaker)
    ans = ev
end

function eval_melee(a::Spud, b::Spud, tiebreaker1::Int64 = 0, tiebreaker2::Int64 = 0)::Int64
    comp_p = compare_int_list(spud_p_seq(a), spud_p_seq(b), tiebreaker1)
    comp_r = compare_int_list(spud_r_seq(a), spud_r_seq(b), tiebreaker1)
    comp_s = compare_int_list(spud_s_seq(a), spud_s_seq(b), tiebreaker1)
    ev = 4 * comp_p + 3 * comp_r + 2 * comp_s
    return sign(ev + (1-abs(ev))*tiebreaker2)
end



eval_melee (generic function with 3 methods)

In [6]:


function apply_ability(a::Spud, b::Spud)::Array{Spud}
    if a.a1 == ability_none
        return [a, b]
    else
        return ability_fx[a.a1](a, b)
    end
end


function eval_battle(a::Spud, b::Spud)::Int64
    utb = compare_int_list(spud_utb_seq(a), spud_utb_seq(b), 0) # universal tiebreaker
    if utb == 0
        return 0
    end
    a_ability_first = compare_int_list([a.a1, a.s], [b.a1, b.s], utb)
    if a_ability_first ==1
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
    end
    if a_ability_first == -1
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
    end
    #println(a)
    #println(b)
    a_finds = eval_finds(a, b, utb)==1
    b_finds = eval_finds(b, a, utb)==1
    melee_win = eval_melee(a, b, 0, 0)
    luck_win = compare_int_list([a.l, melee_win], [b.l, -melee_win], 0)
    if melee_win ==1 && luck_win ==1
        return 1
    end
    if melee_win == -1 && luck_win == -1
        return -1
    end
    if a_finds && b_finds
        return melee_win
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        return luck_win
    end
end

eval_battle (generic function with 1 method)

In [7]:
tab = CSV.read("census_yob2022_names.txt", DataFrame, header = false)
names = tab.Column1
adjectives = CSV.read("adjectives.csv", DataFrame)
nouns = CSV.read("nouns.csv", DataFrame)
jobs = CSV.read("jobs.csv", DataFrame)


const mult_noun = 1
const mult_adj = 2
const mult_job = 3


function random_name_and_stat(ability::Int64 = ability_none)::Spud
    vp = [0, 0, 0, 0, 0, 0]
    #nametype = rand([1,1,1,1,1,2,2,2,3])
    name = ""
    noun = ""
    adj = ""
    noun_i = rand(1:nrow(nouns))
    noun = nouns[noun_i, :noun]
    vp[1] = vp[1] + nouns[noun_i, :H] * mult_noun
    vp[2] = vp[2] + nouns[noun_i, :F] * mult_noun
    vp[3] = vp[3] + nouns[noun_i, :L] * mult_noun
    vp[4] = vp[4] + nouns[noun_i, :P] * mult_noun
    vp[5] = vp[5] + nouns[noun_i, :R] * mult_noun
    vp[6] = vp[6] + nouns[noun_i, :S] * mult_noun
    adj_i = rand(1:nrow(adjectives))
    if ability == ability_none
        adj = adjectives[adj_i, :adjective]
        vp[1] = vp[1] + adjectives[adj_i, :H] * mult_adj
        vp[2] = vp[2] + adjectives[adj_i, :F] * mult_adj
        vp[3] = vp[3] + adjectives[adj_i, :L] * mult_adj
        vp[4] = vp[4] + adjectives[adj_i, :P] * mult_adj
        vp[5] = vp[5] + adjectives[adj_i, :R] * mult_adj
        vp[6] = vp[6] + adjectives[adj_i, :S] * mult_adj
    else
        adj = ability_name[ability]
    end
    job_i = rand(1:nrow(jobs))
    job = jobs[job_i, :job]
    vp[1] = vp[1] + jobs[job_i, :H] * mult_job
    vp[2] = vp[2] + jobs[job_i, :F] * mult_job
    vp[3] = vp[3] + jobs[job_i, :L] * mult_job
    vp[4] = vp[4] + jobs[job_i, :P] * mult_job
    vp[5] = vp[5] + jobs[job_i, :R] * mult_job
    vp[6] = vp[6] + jobs[job_i, :S] * mult_job
    name = string(adj, " ", job, " ", noun)
    Spud(name, vp[1], vp[2], vp[3], vp[4], vp[5], vp[6], ability_none)
end

random_name_and_stat (generic function with 2 methods)

In [8]:
function rand_rename(a::Spud, n_tries::Int = 100)::Spud
    best_score = 0.0
    best_b = random_name_and_stat(a.a1)
    for ii in 1:n_tries
        b = random_name_and_stat(a.a1)
        b_norm = sqrt(b.h^2 + b.f^2 + b.l^2 + b.p^2 + b.r^2 + b.s^2)
        score = (a.h * b.h + a.f * b.f + a.l * b.l + a.p * b.p + a.r * b.r + a.s * b.s)/b_norm
        if score > best_score
            best_score = score
            best_b = b
        end
    end
    return Spud(best_b.name, a.h, a.f, a.l, a.p, a.r, a.s, a.a1)
end

rand_rename (generic function with 2 methods)

In [9]:
function eval_battle_list(a::Spud, bs::Array{Spud})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function eval_battle_list2(a::Spud, bs::Array{Spud}, w::Vector{Float64})::AbstractFloat
    score = 0.0
    for ii in 1:length(bs)
        score = score + w[ii] * eval_battle(a, bs[ii])
    end
    return score
end

function spuds_to_df(as::Array{Spud})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ps = Array{Int}(undef, length(as))
    rs = Array{Int}(undef, length(as))
    ss = Array{Int}(undef, length(as))
    a1s = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ps[ii] = as[ii].p
        rs[ii] = as[ii].r
        ss[ii] = as[ii].s
        a1s[ii] = as[ii].a1
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, p = ps, r = rs, s = ss, a1 = a1s)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

# for legacy dfs without abilities
function df_to_spuds0(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], ability_none)
    end
    return as
end

function df_to_spuds(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], df[i, :a1])
    end
    return as
end



df_to_spuds (generic function with 1 method)

## Get spuds with cost 10 and filter nondominated

In [10]:
function get_library(cost, n_init = 10000)
    ffs = Array{Spud}(undef, n_init)
    ff_i = 0
    hrange = MNV:MXV
    frange = MNV:MXV
    lrange = MNV:MXV
    prange = MNV:MXV
    rrange = MNV:MXV
    srange = MNV:MXV
    for h in hrange
        for f in frange
            for l in lrange
                for p in prange
                    for r in rrange
                        for s in srange
                            if (h+f+l+p+r+s == cost)
                                ff = Spud("", h,f,l,p,r,s,999)
                                ff_i += 1
                                ffs[ff_i] = ff
                            end
                        end
                    end                        
                end                        
            end
        end
    end
    ffs = ffs[1:ff_i]
    return unique(ffs)
end

get_library (generic function with 2 methods)

In [11]:
lib0 = get_library(10)
lib0

126-element Vector{Spud}:
 Spud("", 1, 1, 1, 1, 1, 5, 999)
 Spud("", 1, 1, 1, 1, 2, 4, 999)
 Spud("", 1, 1, 1, 1, 3, 3, 999)
 Spud("", 1, 1, 1, 1, 4, 2, 999)
 Spud("", 1, 1, 1, 1, 5, 1, 999)
 Spud("", 1, 1, 1, 2, 1, 4, 999)
 Spud("", 1, 1, 1, 2, 2, 3, 999)
 Spud("", 1, 1, 1, 2, 3, 2, 999)
 Spud("", 1, 1, 1, 2, 4, 1, 999)
 Spud("", 1, 1, 1, 3, 1, 3, 999)
 Spud("", 1, 1, 1, 3, 2, 2, 999)
 Spud("", 1, 1, 1, 3, 3, 1, 999)
 Spud("", 1, 1, 1, 4, 1, 2, 999)
 ⋮
 Spud("", 3, 1, 3, 1, 1, 1, 999)
 Spud("", 3, 2, 1, 1, 1, 2, 999)
 Spud("", 3, 2, 1, 1, 2, 1, 999)
 Spud("", 3, 2, 1, 2, 1, 1, 999)
 Spud("", 3, 2, 2, 1, 1, 1, 999)
 Spud("", 3, 3, 1, 1, 1, 1, 999)
 Spud("", 4, 1, 1, 1, 1, 2, 999)
 Spud("", 4, 1, 1, 1, 2, 1, 999)
 Spud("", 4, 1, 1, 2, 1, 1, 999)
 Spud("", 4, 1, 2, 1, 1, 1, 999)
 Spud("", 4, 2, 1, 1, 1, 1, 999)
 Spud("", 5, 1, 1, 1, 1, 1, 999)

In [12]:
function get_payoffs(env::Array{Spud})::Array{Int64}
    n_nash = length(env)
    payoffs = Array{Int64}(undef, (n_nash, n_nash))
    for i in 1:n_nash
        for j in 1:n_nash
            payoffs[i, j] = eval_battle(env[i], env[j])
        end
    end
    return payoffs
end

get_payoffs (generic function with 1 method)

In [13]:
function filter_nondominated2(as::Array{Spud})::Array{Spud}
    mat = get_payoffs(as);
    isDominated = zeros(Int64, length(as));
    for i in 1:length(as)
        v = mat[i, :]
        bv = ones(Int64, length(as))
        for j in 1:length(as)
            bv = bv .* (mat[:, j] .>= v[j])
        end
        if sum(bv) > 1
            isDominated[i] = 1
        end
    end
    return as[isDominated .== 0]
end

filter_nondominated2 (generic function with 1 method)

In [14]:
lib=filter_nondominated2(lib0)
for ff in lib
    print(" ")
    print(100000 * ff.h + 10000 * ff.f + 1000 * ff.l + 100 * ff.p + 10 * ff.r + ff.s)
end

 111124 111133 111142 111214 111223 111232 111241 111313 111322 111331 111412 111421 111511 112114 112123 112132 112141 112213 112222 112231 112312 112321 112411 113113 113122 113131 113212 113221 113311 114112 114121 114211 121114 121123 121132 121213 121222 121231 121312 121321 121411 122113 122122 122212 122221 122311 123112 123211 131113 131122 131212 131221 131311 132112 132211 141112 141211 212113 212122 212131 212212 212221 212311 213112 213121 213211 214111 222112 222211 312112 312121 312211 313111 412111

In [15]:
payoffs = get_payoffs(lib)

74×74 Matrix{Int64}:
  0  -1  -1  -1  -1  -1  -1   1  -1  …  -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   0  -1  -1  -1  -1  -1  -1   1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   0  -1  -1  -1  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   1   0  -1  -1  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   1   1   0  -1  -1  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   1   1   1   0  -1  -1  -1  …  -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1   1   1   1   1   0  -1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 -1   1   1   1   1   1   1   0  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1  -1   1   1   1   1   1   1   0     -1  -1  -1  -1  -1  -1  -1  -1  -1
  1   1  -1   1   1   1   1   1   1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 -1  -1   1   1  -1   1   1   1   1  …  -1  -1  -1  -1  -1  -1  -1  -1  -1
  1  -1  -1   1   1  -1   1   1   1     -1  -1  -1  -1  -1  -1  -1  -1  -1
 -1  -1  -1   1  -1  -1   1   1  -1     -1  -1  -1  -1  -1  -1  -1  -1  -1
  ⋮ 

In [139]:
s2ind = Dict(lib[i] => i for i in 1:length(lib));

In [69]:
s2ind[lib[5]]

5

## Battle code

In [152]:
mutable struct BattleView
    round::Int64
    score::Int64
    own::Array{Spud}
    obs::Array{Spud}
end

mutable struct AiPlayer
    create::Function
end

In [153]:
function initialize_game()::Array{BattleView}
    bvs = [BattleView(0,0, [], []), BattleView(0,0, [], [])]
    return bvs
end

initialize_game()

2-element Vector{BattleView}:
 BattleView(0, 0, Spud[], Spud[])
 BattleView(0, 0, Spud[], Spud[])

In [154]:
mod(2, 2)

0

In [155]:
lib[end]

Spud("", 4, 1, 2, 1, 1, 1, 999)

In [156]:
function spud_to_str(ff::Spud)::String
    return string(100000 * ff.h + 10000 * ff.f + 1000 * ff.l + 100 * ff.p + 10 * ff.r + ff.s)
end

spud_to_str (generic function with 1 method)

In [157]:
function eval_beat_n(ff::Spud, opps::Array{Spud})::Int64
    flag = true
    count = 0
    while flag && count < length(opps)
        b = opps[end - count]
        if eval_battle(ff, b) == 1
            count += 1
        else
            flag = false
        end
    end
    return count
end

eval_beat_n (generic function with 1 method)

In [158]:
# occurs after spud creation
function play_combat(game::Array{BattleView}, verbose::Bool = false)::Array{BattleView}
    round = game[1].round
    if mod(round, 2)==1
        ff = game[1].own[end]
        opps = game[2].own
    else
        ff = game[2].own[end]
        opps = game[1].own
    end
    count = eval_beat_n(ff, opps)
    if verbose
        print(spud_to_str(ff))
        print(" beats ")
        println([spud_to_str(b) for b in opps[(end-count+1):end]])
    end
    if mod(round, 2)==1
        game[1].score = count
    else
        game[2].score = count
    end
    return game
end

play_combat (generic function with 2 methods)

In [159]:
function play_add_spud(game::Array{BattleView}, player::Int64, ff::Spud, verbose::Bool = false)::Array{BattleView}
    round = game[1].round + 1
    game[1].round = round
    game[2].round = round
    game[player].own = append!(game[player].own, [ff])
    game[3-player].obs = game[player].own
    if verbose
        print("Round ")
        println(round)
        print("Score to match or beat: ")
        println(max(game[1].score, game[2].score))
        print("Player ")
        print(player)
        print(" adds #")
        print(["A", "B"][player])
        print(length(game[player].own))
        print(" ")
        println(spud_to_str(ff))
    end
    return game
end



play_add_spud (generic function with 2 methods)

In [160]:
# opps = lib[[5,4,3,2,1]]
# ff = lib[3]
# opps

In [161]:
# ff

In [162]:
# team_a = lib[9:11]
# team_b = lib[[1,10]]
# game = [BattleView(5, 2, team_a, team_b), BattleView(5, 2, team_b, team_a)]

In [163]:
# team_a

In [164]:
# team_b

In [165]:
# play_combat(game, true)

In [166]:
# play_add_spud(game, 2, lib[20], true)

In [167]:
# play_combat(game, true)

In [168]:
# play_add_spud(game, 1, lib[19], true)

In [169]:
# play_combat(game, true)

In [170]:


function play_is_game_over(game::Array{BattleView})::Bool
    round = game[1].round
    if mod(round, 2)==1
        if game[1].score < game[2].score
            return true
        end
    else
        if game[2].score < game[1].score
            return true
        end        
    end
    return false
end

play_is_game_over (generic function with 1 method)

In [283]:
function beatn_counts(payoffs::Array{Int64}, s2ind::Dict{Spud, Int64}, as::Array{Spud}, n::Int64 = 8)::Array{Int64}
    ainds = [s2ind[a] for a in as]
    bv = zeros(Int64, length(lib))
    counts = zeros(Int64, n)
    for i in 1:min(length(as), n)
        row = payoffs[ainds[end - i + 1], :]
        bv[row .== -1 .&& bv .== (i-1)] .= i
        counts[i] = sum(bv .== i)
    end
    return counts
end

beatn_counts (generic function with 6 methods)

In [284]:
# as = [rand(lib) for i in 1:3]
# as

In [285]:
# beatn_counts(payoffs, s2ind, as)

In [286]:
# lib[findall([eval_battle_list(ff, as) for ff in lib] .== 3)]

In [287]:
function ai0_create(a::BattleView, lib::Array{Spud})::Spud
    return rand(lib)
end

function highest_beat_n(ops::Array{Spud}, lib::Array{Spud})::Array{Spud}
    ebs = [eval_beat_n(ff, ops) for ff in lib]
    cands = lib[ebs .== maximum(ebs)]
    return cands
end

# aggressive play
function ai1_create(a::BattleView, lib::Array{Spud})::Spud
    ops = a.obs
    cands = highest_beat_n(ops, lib)
    return rand(cands)
end


ai1_create (generic function with 2 methods)

In [288]:
# defensive play
function ai2_create(a::BattleView, 
        lib::Array{Spud}, payoffs::Array{Int64}, s2ind::Dict{Spud, Int64}, 
        aggro::Bool = false)::Spud
    ops = a.obs
    as = a.own
    if length(ops) == 0
        sp = [sum(payoffs[i, :]) for i in 1:length(lib)]
        cands = lib[sp .== maximum(sp)]
        return rand(cands)
    end
    to_beat = eval_beat_n(ops[end], a.own)
    scores = [eval_beat_n(ff, ops) for ff in lib]
    if aggro
        cands = lib[findall(scores .== maximum(scores))]
    else
        cands = lib[findall(scores .>= to_beat)]
    end
    if length(cands) == 0
        cands = lib[findall(scores .== maximum(scores))]
    end
    bscores = zeros(Float64, length(cands))    
    for i in 1:length(cands)
        ff = cands[i]
        a2s = copy(as)
        append!(a2s, [ff])
        bc = beatn_counts(payoffs, s2ind, a2s)
        nb = sum(bc .> 0)
        if nb == 0
            bscores[i] = 0
        else
            bscores[i] = nb + (bc[nb] - 1)/length(lib)
        end
    end
    fcands = cands[bscores .== minimum(bscores)]
    scores = [eval_beat_n(ff, ops) for ff in fcands]
    fcands2 = fcands[scores .== maximum(scores)]
    return rand(fcands2)
end


ai2_create (generic function with 3 methods)

In [289]:
function input_spud()::Spud
    s = readline()
    return stats_to_spud(s)
end

function stats_to_spud(s::String)::Spud
    v = Array{Int64}(undef, 6)
    for i in 1:6
        v[i] = convert(Int64, s[i]) - 48
    end
    return Spud("", v[1], v[2], v[3], v[4], v[5], v[6], 999)
end

function player_create(a::BattleView)::Spud
    return input_spud()
end

player_create (generic function with 1 method)

In [290]:
function play_game(players::Array{AiPlayer}, verbose::Bool = true)::Int64
    ai1 = players[1]
    ai2 = players[2]
    ais = players
    game = initialize_game()
    while !play_is_game_over(game)
        # play a round
        round = game[1].round
        if mod(round, 2)==0
            ff = ais[1].create(game[1])
            game = play_add_spud(game, 1, ff, verbose)
        else
            ff = ais[2].create(game[2])
            game = play_add_spud(game, 2, ff, verbose)
        end
        if round > 0
            game = play_combat(game, verbose)
        else
            if verbose
                println("First round: no combat.")
            end
        end
    end
    if verbose
        println("Game history:")
        for i in 1:length(game[1].own)
            print(spud_to_str(game[1].own[i]))
            print(" ")
            if i <= length(game[2].own)
                println(spud_to_str(game[2].own[i]))
            end
        end
    end
    if game[1].score > game[2].score
        return 1
    else
        return -1
    end
end

play_game (generic function with 2 methods)

In [291]:
function play_game_n(players::Array{AiPlayer}, n::Int64)
    verbose = true
    ai1 = players[1]
    ai2 = players[2]
    ais = players
    game = initialize_game()
    for i in 1:n
        # play a round
        round = game[1].round
        if mod(round, 2)==0
            ff = ais[1].create(game[1])
            game = play_add_spud(game, 1, ff, verbose)
        else
            ff = ais[2].create(game[2])
            game = play_add_spud(game, 2, ff, verbose)
        end
        if round > 0
            game = play_combat(game, verbose)
        else
            println("First round: no combat.")
        end
    end
    if verbose
        println("Game history:")
        for i in 1:length(game[1].own)
            print(spud_to_str(game[1].own[i]))
            print(" ")
            if i <= length(game[2].own)
                println(spud_to_str(game[2].own[i]))
            end
        end
    end
end

play_game_n (generic function with 1 method)

In [292]:
ai1 = AiPlayer(x -> ai1_create(x, lib))
ai2 = AiPlayer(x -> ai1_create(x, lib))
ais = [ai1, ai2]
play_game(ais, true)
#play_game_n(ais, 1000)

Round 1
Score to match or beat: 0
Player 1 adds #A1 112222
First round: no combat.
Round 2
Score to match or beat: 0
Player 2 adds #B1 112312
112312 beats ["112222"]
Round 3
Score to match or beat: 1
Player 1 adds #A2 113122
113122 beats ["112312"]
Round 4
Score to match or beat: 1
Player 2 adds #B2 111313
111313 beats ["112222", "113122"]
Round 5
Score to match or beat: 2
Player 1 adds #A3 111412
111412 beats ["112312", "111313"]
Round 6
Score to match or beat: 2
Player 2 adds #B3 214111
214111 beats ["112222", "113122", "111412"]
Round 7
Score to match or beat: 3
Player 1 adds #A4 222112
222112 beats ["112312", "111313", "214111"]
Round 8
Score to match or beat: 3
Player 2 adds #B4 121123
121123 beats ["112222", "113122", "111412", "222112"]
Round 9
Score to match or beat: 4
Player 1 adds #A5 121321
121321 beats ["111313", "214111", "121123"]
Game history:
112222 112312
113122 111313
111412 214111
222112 121123
121321 

-1

In [298]:
ai1 = AiPlayer(x -> ai1_create(x, lib))
ai2 = AiPlayer(x -> ai2_create(x, lib, payoffs, s2ind, false))
ais = [ai1, ai2]
#play_game(ais, true)
mean([play_game(ais, false) for i in 1:100])
#play_game_n(ais, 1000)

-0.38

In [299]:
ai1 = AiPlayer(x -> ai2_create(x, lib, payoffs, s2ind))
ai2 = AiPlayer(x -> ai2_create(x, lib, payoffs, s2ind))
ais = [ai1, ai2]
play_game(ais, true)
#play_game_n(ais, 1000)

Round 1
Score to match or beat: 0
Player 1 adds #A1 112222
First round: no combat.
Round 2
Score to match or beat: 0
Player 2 adds #B1 112222
112222 beats String[]
Round 3
Score to match or beat: 0
Player 1 adds #A2 214111
214111 beats ["112222"]
Round 4
Score to match or beat: 1
Player 2 adds #B2 121321
121321 beats ["112222", "214111"]
Round 5
Score to match or beat: 2
Player 1 adds #A3 111133
111133 beats ["112222", "121321"]
Round 6
Score to match or beat: 2
Player 2 adds #B3 121213
121213 beats ["214111", "111133"]
Round 7
Score to match or beat: 2
Player 1 adds #A4 313111
313111 beats ["112222", "121321", "121213"]
Round 8
Score to match or beat: 3
Player 2 adds #B4 213121
213121 beats ["111133", "313111"]
Game history:
112222 112222
214111 121321
111133 121213
313111 213121


1

In [56]:
ai1 = AiPlayer(ai1_create)
ai2 = AiPlayer(player_create)
ais = [ai1, ai2]
game = play_game(ais, true)

Round 1
Score to match or beat: 0
Player 1 adds #A1 132211
First round: no combat.
stdin> 313111
Round 2
Score to match or beat: 0
Player 2 adds #B1 313111
313111 beats String[]
Round 3
Score to match or beat: 0
Player 1 adds #A2 131221
131221 beats ["313111"]
stdin> 121132
Round 4
Score to match or beat: 1
Player 2 adds #B2 121132
121132 beats ["132211", "131221"]
Round 5
Score to match or beat: 2
Player 1 adds #A3 213112
213112 beats ["313111", "121132"]
stdin> 113212
Round 6
Score to match or beat: 2
Player 2 adds #B3 113212
113212 beats ["132211", "131221", "213112"]
Round 7
Score to match or beat: 3
Player 1 adds #A4 111412
111412 beats ["121132", "113212"]
Game history:
132211 313111
131221 121132
213112 113212
111412 

-1

In [55]:
highest_beat_n([stats_to_spud(s) for s in ["113311", "213211", "213112", "121132"]])

9-element Vector{Spud}:
 Spud("", 1, 1, 3, 1, 2, 2, 999)
 Spud("", 1, 1, 4, 1, 1, 2, 999)
 Spud("", 1, 2, 1, 2, 1, 3, 999)
 Spud("", 1, 2, 1, 2, 2, 2, 999)
 Spud("", 1, 2, 1, 3, 1, 2, 999)
 Spud("", 1, 2, 2, 1, 1, 3, 999)
 Spud("", 1, 2, 2, 2, 1, 2, 999)
 Spud("", 1, 2, 3, 1, 1, 2, 999)
 Spud("", 1, 3, 1, 2, 1, 2, 999)